In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: ml_workspace_p
Azure region: eastus
Subscription id: 76be00d8-3bef-43a6-89ba-2a12016e7f8c
Resource group: resourcegroup_p


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Compute-Inst-DS11-P" #"compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

# get status of the cluster
print(cpu_cluster.get_status().serialize())

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2022-04-22T19:18:14.328544+00:00', 'createdBy': {'userObjectId': 'f84c0cef-c750-41f6-8f8c-8ff6807766a4', 'userTenantId': 'd1ce8b91-bab1-49fc-937d-a779f008ae95', 'userName': None}, 'modifiedTime': '2022-05-29T13:02:29.127109+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS11_V2'}


In [3]:
# !pip3 install azureml.widgets
# pip install azureml-sdk[notebooks]
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler

parameter_space = {"C": choice(1), "max_iter": choice(100)}
ps = RandomParameterSampling(parameter_space = parameter_space)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = ".",
                            script='train.py',
                            compute_target=cluster_name,
                            environment = sklearn_env)


# est = SKLearn(source_directory=".", 
#               compute_target=cluster_name, 
#               vm_priority='lowpriority', 
#               entry_script='train.py')


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
## ScriptRunConfig() should be passed to the run_config parameter instead of the estimator
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal= PrimaryMetricGoal("MAXIMIZE"),
                                     max_total_runs=20,
                                     max_concurrent_runs=4,
                                     policy=policy)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

#hyperdrive_run.wait_for_completion(show_output=True)

#assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
# best_run = hyperdrive_run.get_best_run_by_primary_metric()
# best_run

best_model_metrics = hyperdrive_run.get_metrics()

print("Best run metrics :",best_model_metrics)



#hyperdrive_run.get_children_sorted_by_primary_metric(top=1)

#joblib.dump(best_run, 'hd_bestrun_model.joblib')

Best run metrics : {}


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

In [8]:

!python train.py --C=1 --max_iter=100

Attempted to log scalar metric Regularization Strength::
1.0
Attempted to log scalar metric Max iterations::
100
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
Attempted to log scalar metric Accuracy:
0.9112797167425392
